## Using Python to Integrate MongoDB Data into an ETL Process
Modern Data Warehousing and Analytics solutions frequently use languages like Python or Scala to extract data from numerous sources, including relational database management systems, NoSQL database systems, real-time streaming endpoints and Data Lakes. These languages can then be used to perform many types of transformation before then loading the data into a variety of destinations including file systems and data warehouses. This data can then be consumed by data scientists or business analysts.

In this lab you will build upon the **Northwind_DW2** dimensional database from Lab 3; however, you will be integrating new data sourced from an instance of MongoDB. The new data will be concerned with new business processes; inventory and purchasing. You will continue to interact with both the source systems (MongoDB and MySQL), and the destination system (the Northwind_DW2 data warehouse) from a remote client running Python (Jupyter Notebooks). 

Just as in Lab 3, you will fetch data into Pandas DataFrames, perform all the necessary transformations in-memory on the client, and then push the newly transformed DataFrame to the RDBMS data warehouse using a Pandas function that will create the table and fill it with data with a single operation.

### Prerequisites:
This notebook uses the PyMongo database connectivity library to connect to MySQL databases; therefore, you must have first installed that libary into your python environment by executing the following command in a Terminal window.

- `python -m pip install pymongo[srv]`

#### Import the Necessary Libraries

In [1]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 1.4.39
Running PyMongo Version: 4.6.3


#### Declare & Assign Connection Variables for the MongoDB Server, the MySQL Server & Databases with which You'll be Working 

In [3]:
mysql_args = {
    "uid" : "root",
    "pwd" : "Passw0rd123",
    "hostname" : "localhost",
    "dbname" : "northwind_dw2"
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "",
    "password" : "password",
    "cluster_name" : "cluster_name",
    "cluster_subnet" : "xxxxx",
    "cluster_location" : "local", # "local"
    "db_name" : "northwind_purchasing"
}

#### Define Functions for Getting Data From and Setting Data Into Databases

In [4]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

#### Populate MongoDB with Source Data
You only need to run this cell once; however, the operation is *idempotent*.  In other words, it can be run multiple times without changing the end result.

In [6]:
client = get_mongo_client(**mongodb_args)

# Gets the path of the Current Working Directory for this Notebook,
# and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"customers" : 'classicmodels_customers.json'}

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)         

### 1.0. Create and Populate the New Dimension Tables
#### 1.1. Extract Data from the Source MongoDB Collections Into DataFrames

In [10]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "customers"

df_customers = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_customers.head(2)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800.0


#### 1.2. Lookup the Invoice Date Keys from the Date Dimension Table.
Here we see an example of where a dimension cross-references another dimension; the Date dimension.  The Date dimension is a classic example of a **Role-Playing dimension**. Dates in-and-of themselves are universal; however, when applied to specific events they take on the identity of those events. Here, the *dim_date* table takes on the identity of *dim_invoice_date* to supply invoice date keys to the *dim_invoices* table.

##### 1.2.1. Get the Data from the Date Dimension Table.
First, fetch the Surrogate Primary Key (date_key) and the Business Key (full_date) from the Date Dimension table using the **get_sql_dataframe()** function. Be certain to cast the **full_date** column to the **datetime64[ns]** data type using the **.astype()** function that is native to Pandas DataFrame columns. Also, extract the **date** portion using the **.dt.date** attribute of the **datetime64[ns]** datatype.

In [8]:
sql_dim_date = "SELECT date_key, full_date FROM classicmodels_dw2.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02


##### 1.2.2. Lookup the Surrogate Primary Key (date_key) that Corresponds to the invoice_date Column

In [ ]:
df_dim_invoice_date = df_dim_date.rename(columns={"date_key" : "invoice_date_key", "full_date" : "invoice_date"})
df_invoices.invoice_date = df_invoices.invoice_date.astype('datetime64[ns]').dt.date
df_invoices = pd.merge(df_invoices, df_dim_invoice_date, on='invoice_date', how='left')
df_invoices.drop(['invoice_date'], axis=1, inplace=True)
df_invoices.head(2)

#### 1.3. Perform Any Necessary Transformations to the DataFrames

In [11]:
# 1. Rename the "id" column to reflect the entity as it will serve as the business key for lookup operations
#df_customers.rename(columns={"id":"invoice_id"}, inplace=True)

# 2. Since there are no values in the 'due_date' column go ahead and drop it.
df_customers.drop(['phone','addressLine2','state'], axis=1, inplace=True)

# 3. Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_customers.insert(0, "customer_key", range(1, df_invoices.shape[0]+1))
df_customers.head(2)

,customer_key,customerNumber,customerName,contactLastName,contactFirstName,addressLine1,city,postalCode,country,salesRepEmployeeNumber,creditLimit
0,1,103,Atelier graphique,Schmitt,Carine,"54, rue Royale",Nantes,44000,France,1370.0,21000.0
1,2,112,Signal Gift Stores,King,Jean,8489 Strong St.,Las Vegas,83030,USA,1166.0,71800.0


#### 1.3. Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables

Here we will call our **set_dataframe( )** function to create each dimension table. This function expects a number of parameters including the usual connection information (e.g., user_id, password, MySQL server name and database), the *table_name* we need to assign to the table, the *pandas DataFrame* we crafted to define & populate the table, the *name* of the column we wish to designate as the *primary_key* column, and finally, the database operation (insert or update). 

In [12]:
dataframe = df_customers
table_name = 'dim_customers'
primary_key = 'customer_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

#### 1.4. Validate that the New Dimension Tables were Created.

In [14]:
sql_customers = "SELECT * FROM classicmodels_dw2.dim_customers;"
df_dim_customers = get_sql_dataframe(sql_customers, **mysql_args)
df_dim_customers.head(2)

,customer_key,customerNumber,customerName,contactLastName,contactFirstName,addressLine1,city,postalCode,country,salesRepEmployeeNumber,creditLimit
0,1,103,Atelier graphique,Schmitt,Carine,"54, rue Royale",Nantes,44000,France,1370.0,21000.0
1,2,112,Signal Gift Stores,King,Jean,8489 Strong St.,Las Vegas,83030,USA,1166.0,71800.0


### 2.0. Create and Populate the New Fact Tables
#### 2.1. Extract Data from the Source MongoDB Collections Into DataFrames

In [15]:
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "customers"

df_fact_pos = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_fact_pos.head(2)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800.0


In [16]:
# TODO: Extract data for your new "Inventory Transactions" Fact Table
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "customers"

df_fact_inventory = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_fact_inventory.head(2)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800.0


#### 2.3. Lookup the Primary Keys from the Dimension Tables
**Foreign key relationships** must be established between each newly-crafted **Fact table** and each related **Dimension table**.

##### 2.3.1. First, fetch the Surrogate Primary Key and the Business Key from each Dimension table.

In [17]:
sql_dim_customers = "SELECT customer_key, customerNumber FROM classicmodels_dw2.dim_customers;"
df_dim_customers = get_sql_dataframe(sql_dim_customers, **mysql_args)
df_dim_customers.head(2)

,customer_key,customerNumber
0,1,103
1,2,112


##### 2.3.2. Next, use Business Keys to lookup corresponding Surrogate Primary Keys in the Dimension tables
**2.3.2.1. Inventory Fact table.**  The inventory fact table contains a reference to the *dim_products* table by way of the *product_id* column that enables us to lookup the corresponding *product_key*.

In [18]:
# TODO: Merge the "fact_inventory" and "dim_products" dataframes on the 'product_id' to
# get the 'product_key'. Then drop the 'product_id' column and display the results.
df_fact_inventory = pd.merge(df_fact_inventory, df_dim_customers, on='customerNumber', how='inner')
df_fact_inventory.drop(['customerNumber'], axis=1, inplace=True)
df_fact_inventory.head(2)

,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit,customer_key
0,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0,1
1,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800.0,2


ValueError: cannot insert customer_key, already exists

#### 2.4. Perform Any Necessary Transformations to the DataFrames

In [22]:
# Rename the Foreign Key Columns
column_name_map = {}

df_fact_inventory.rename(columns=column_name_map, inplace=True)

# Reorder the Columns
ordered_columns = []

df_fact_inventory = df_fact_inventory[ordered_columns]
df_fact_inventory.head(2)

""
0
1


#### 2.5. Load Newly Transformed MongoDB Data into the classicmodels_DW2 Data Warehouse

In [ ]:
dataframe = df_fact_pos
table_name = 'fact_purchase_orders'
primary_key = 'fact_purchase_order_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [ ]:
# TODO: Upload the "Inventory Transaction" dataframe to create the new "fact_inventory_transactions" fact table. 
dataframe = df_fact_inventory
table_name = 'fact_inventory_transactions'
primary_key = 'fact_inventory_transaction_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

#### 2.6. Validate that the New Fact Tables were Created

In [ ]:
# TODO: Validate the correctness of the new "Purchase Orders" fact table.
sql_purchase_orders = "SELECT * FROM northwind_dw2.fact_purchase_orders;"
df_fact_purchase_orders = get_sql_dataframe(sql_purchase_orders, **mysql_args)
df_fact_purchase_orders.head(2)

In [ ]:
# TODO: Validate the correctness of the new "Inventory Transactions" fact table.
sql_inventory_transactions = "SELECT * FROM northwind_dw2.fact_inventory_transactions;"
df_fact_inventory_transactions = get_sql_dataframe(sql_inventory_transactions, **mysql_args)
df_fact_inventory_transactions.head(2)

### 3.0. Demonstrate that the New Data Warehouse Exists and Contains the Correct Data
To demonstrate the viability of your solution, author a SQL SELECT statement that returns:
- Each Company’s Name
- The total amount of the purchase order detail quantity associated with each company
- The total amount of the purchase order detail unit cost associated with each company

**NOTE:** *Remember that a string typed variable whose value is contained by triple-quotes (""" ... """) can preserve multi-line formatting, and that a string variable has an intrinsic **.format()** function that accepts ordered parameters that will replace tokens (e.g., {0}) in the formatted string.*  

In [ ]:
sql_purchase_orders = """
    SELECT s.company AS 'Company'
        , FORMAT(SUM(po.po_detail_quantity),0) AS 'Total Quantity'
        , FORMAT(SUM(po.po_detail_unit_cost),2) AS 'Total Unit Cost'
    FROM northwind_dw2.fact_purchase_orders AS po
    INNER JOIN northwind_dw2.dim_suppliers AS s
    ON po.supplier_key = s.supplier_key
    GROUP BY s.company
    ORDER BY 'Total Unit Cost' DESC;
"""

In [ ]:
df_fact_purchase_orders = get_sql_dataframe(sql_purchase_orders, **mysql_args)
df_fact_purchase_orders

### 3.1 Extra Credit: Author a Query that Returns the Inventory Transaction Data
- Each Product Category
- Each TransactionType
- The total number of transactions associated with each Product Category and Transaction Type
- The total amount (quantity) associated with each Product Category and Transaction Type

In [ ]:
sql_inventory_transactions = """
    SELECT p.category AS 'Product Category'
        , it.inventory_transaction_type AS 'Transaction Type'
        , COUNT(it.inventory_transaction_id) 'Number of Transactions'
        , FORMAT(SUM(it.quantity),0) AS 'Total Quantity'
    FROM northwind_dw2.fact_inventory_transactions AS it
    INNER JOIN northwind_dw2.dim_products AS p
    ON p.product_key = it.product_key
    GROUP BY p.category
        , it.inventory_transaction_type
    ORDER BY 'Number of Transactions' DESC
        , 'Total Quantity' DESC;
"""

In [ ]:
df_fact_inventory_transactions = get_sql_dataframe(sql_inventory_transactions, **mysql_args)
df_fact_inventory_transactions